# Optimizing Markowitz - L1 plus shrinking the sample covariance matrix towards a one-factor matrix (Ledoit and Wolf, 2003)

## Import functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sklearn
import pypfopt
import itertools
import pypfopt
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.covariance import LedoitWolf

## Data preparation

In [2]:
df = pd.read_csv('Test_data/48_Industry_Portfolios_Daily_Clean.csv', index_col = 0)
df.rename(columns={"Average Value Weighted Returns -- Daily": "date"}, inplace=True)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df = df.loc[(df['date'] >= '1992-12-31') & (df['date'] <= '2014-12-31')]
df = df.set_index('date')
df = df/100
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5542 entries, 1992-12-31 to 2014-12-31
Data columns (total 48 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0    Agric  5542 non-null   float64
 1   Food    5542 non-null   float64
 2   Soda    5542 non-null   float64
 3   Beer    5542 non-null   float64
 4   Smoke   5542 non-null   float64
 5   Toys    5542 non-null   float64
 6   Fun     5542 non-null   float64
 7   Books   5542 non-null   float64
 8   Hshld   5542 non-null   float64
 9   Clths   5542 non-null   float64
 10  Hlth    5542 non-null   float64
 11  MedEq   5542 non-null   float64
 12  Drugs   5542 non-null   float64
 13  Chems   5542 non-null   float64
 14  Rubbr   5542 non-null   float64
 15  Txtls   5542 non-null   float64
 16  BldMt   5542 non-null   float64
 17  Cnstr   5542 non-null   float64
 18  Steel   5542 non-null   float64
 19  FabPr   5542 non-null   float64
 20  Mach    5542 non-null   float64
 21  ElcEq   5542 non-nu

### N * T matrix of T observations on a system of N variables representing T returns on a universe of N stocks (N = 48 assets in each portfolio; T = 250 observation days， P  = 21 observation days, portfolios_set = 252)

In [3]:
observation_days = 250
prediction_days = 21
n_assets = df.shape[1]
n_portfolios = math.floor((len(df)-250)/21)
n_portfolios

252

### Split each group value

In [4]:
def get_all_portfoios(obs_days,pre_days,n):
    obs_data = []
    pre_data = []
    for i in range(0,n):
        obs_data.append(df.iloc[pre_days*i:obs_days + pre_days*i,:])
        pre_data.append(df.iloc[obs_days+pre_days*i:obs_days+pre_days*i+pre_days:,:])
    return obs_data, pre_data

In [5]:
obs_group, pre_group = get_all_portfoios(observation_days, prediction_days, n_portfolios)
print(f"number of observation: {len(obs_group)}\n number of prediction {len(pre_group)}\n shape of observation:{obs_group[0].shape}\n shape of prediction:{pre_group[0].shape}\n\n observation sample \n{obs_group[0]}")

number of observation: 252
 number of prediction 252
 shape of observation:(250, 48)
 shape of prediction:(21, 48)

 observation sample 
             Agric   Food    Soda    Beer    Smoke   Toys    Fun     Books  \
date                                                                         
1992-12-31  0.0111 -0.0081  0.0037 -0.0158  0.0037  0.0027 -0.0035 -0.0026   
1993-01-04 -0.0020 -0.0079 -0.0048 -0.0019 -0.0071  0.0027 -0.0027  0.0041   
1993-01-05 -0.0025 -0.0143 -0.0146 -0.0140 -0.0133  0.0052  0.0026 -0.0005   
1993-01-06  0.0074 -0.0097  0.0139 -0.0009 -0.0306  0.0083  0.0137  0.0081   
1993-01-07 -0.0203 -0.0060 -0.0089 -0.0196  0.0013 -0.0134 -0.0095 -0.0036   
...            ...     ...     ...     ...     ...     ...     ...     ...   
1993-12-20 -0.0072 -0.0024 -0.0125  0.0026 -0.0046 -0.0113 -0.0027  0.0009   
1993-12-21 -0.0081  0.0030 -0.0029  0.0073  0.0094 -0.0187 -0.0063 -0.0021   
1993-12-22 -0.0035  0.0047  0.0348  0.0090  0.0072  0.0069  0.0071  0.0044   
1993-

## Shrinkage estimator of the covariance matrix
### Minimum-variance portfolio (MVP)

In [6]:
def optimal_weights(obs_returns):
    weights = []
    expected_return = []
    volatility = []
    sharpe_ratio = []
    for i in range(len(obs_returns)):
        mu = pypfopt.expected_returns.mean_historical_return(obs_returns[i], returns_data=True, 
                                                compounding=False, frequency=250, log_returns=False)
        cov = pypfopt.risk_models.CovarianceShrinkage(obs_group[i], returns_data=True, 
                                                      frequency=250, log_returns=False)
        S = cov.ledoit_wolf(shrinkage_target='single_factor')
        ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
        raw_weights = ef.min_volatility()
        cleaned_weights = ef.clean_weights()
        weights.append(np.array(list(cleaned_weights.values())))
        performance = ef.portfolio_performance(verbose=False)
        np.array(expected_return.append(performance[0]))
        np.array(volatility.append(performance[1]))
        np.array(sharpe_ratio.append(performance[2]))
    return weights, expected_return, volatility, sharpe_ratio

def optimal_portfolio(optimal_weights, pre_returns):
    variance = []
    sharpe_ratio = []
    for i in range(len(pre_returns)):
        mu = pypfopt.expected_returns.mean_historical_return(pre_returns[i], returns_data=True, 
                                                compounding=False, frequency=21, log_returns=False)
        cov = pypfopt.risk_models.CovarianceShrinkage(pre_returns[i], returns_data=True, 
                                                      frequency=21, log_returns=False)
        S = cov.ledoit_wolf(shrinkage_target='single_factor')
        sharpe_ratio.append(pypfopt.objective_functions.sharpe_ratio(optimal_weights[i], mu, S))
        variance.append(pypfopt.objective_functions.portfolio_variance(optimal_weights[i], S))
    return variance, sharpe_ratio

In [7]:
weights, expected_return, volatility, sharpe_ratio = optimal_weights(obs_group) 

In [8]:
variance_pre, sharpe_ratio_pre = optimal_portfolio(weights, obs_group)

### LWIF Portfolio variances of FF48

In [12]:
var = np.sum(variance_pre)/len(variance_pre)
var

0.0010674064951263496

### LWIF Out-of-sample Sharpe ratio of FF48

In [13]:
sharpe_ratio_ = np.sum(sharpe_ratio_pre)/len(sharpe_ratio_pre)
sharpe_ratio_

0.38438194820553007

### LWIF Turnover of the portfolio strategies.